# Your first scraper
In this project, we will guide you step by step through the process of:

1. creating a self-contained development environment.
1. retrieving some information from an API (a website for computers)
2. leveraging it to scrape a website that does not provide an API
3. saving the output for later processing

Here we query an API for a list of countries and their past leaders. We then extract and sanitize their short bio from Wikipedia. Finally, we save the data to disk.

This task is often the first (coding) step of a datascience project and you will often come back to it in the future.

You will study topics such as *scraping*, *data structures*, *regular expressions*, *concurrency* and *file handling*. We will point out useful resources at the appropriate time. 

Let's dive in!

## Creating a clean environment

Use the [`venv`](https://docs.python.org/3/library/venv.html) command to create a new environment called `wikipedia_scraper_env`.

Activate it.

You will find more info about virtual environments in the course content and on the web.

## API Scraping

### A simple API query
You will start with the basics: how to do a simple request to an [API endpoint](../../2.python/2.python_advanced/05.Scraping/5.apis.ipynb).

You will use the [requests](https://requests.readthedocs.io/en/latest/) external library through the `import` keyword.

NOTE: external libraries need to be installed first. Check their documentation.

Check the [Quickstart](https://requests.readthedocs.io/en/latest/user/quickstart/) section of the Requests documentation to:
1. use the `get()` method to connect to this endpoint: https://country-leaders.onrender.com/status
2. check if the `status_code` is equal to 200, which means OK.
    * if OK, `print()` the `text`` of the response.
    * if not, `print()` the `status_code`. 

Here is the signification of [HTTP status codes](https://en.wikipedia.org/wiki/List_of_HTTP_status_codes).


In [1]:
# import the requests library (1 line)
import requests
# assign the root url (without /status) to the root_url variable for ease of reference (1 line)
root_url = 'https://www.immoweb.be/'

# assign the /status endpoint to another variable called status_url (1 line)
status_url = root_url + '/status'
# query the /status endpoint using the get() method and store it in the req variable (1 line)
status_response = requests.get(status_url)

# check the status_code using a condition and print appropriate messages (4 lines)
if status_response.status_code == 200:
    print('Success!')
else:
    print('Error: ' + str(status_response.status_code))

Error: 500


### Dealing with JSON

[JSON](https://quickref.me/json) is the preferred format to deal with data over the web. You cannot avoid it so you would better get acquainted.

Connect to another endpoint called `/countries` but this time the API will return data in the JSON format. Check what to do in the Requests [Quickstart](https://requests.readthedocs.io/en/latest/user/quickstart/).


In [4]:
# Set the countries_url variable (1 line)
property_url = root_url + 'en/classified/house/for-sale/ghent/9000/10657599'

# query the /countries endpoint using the get() method and store it in the req variable (1 line)
property_response = requests.get(property_url)

# Get the JSON content and store it in the countries variable (1 line)
property = property_response.json()

# display the request's status code and the countries variable (1 line)
print(property_response.status_code)


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

### Cookies anyone?

It looks like the access to this API is restricted...
Query the `/cookie` endpoint and extract the appropriate field to access your cookie.

You will need to use this cookie in each of the following API requests.

In [2]:
# Set the cookie_url variable (1 line)
cookie_url = root_url + '/cookie'

# Query the enpoint, set the cookies variable and display it (2 lines)
cookies = requests.get(cookie_url).cookies
print(cookies)

<RequestsCookieJar[<Cookie __cf_bm=80H9xCdkjBPYpS9tgK9Kpu1LqENtiluxKtldZRVZl0Q-1687773023-0-AXroDSbaYyzGuorxGZXsbiteM8Bz5MZW7f43Kxeyzx2/t2h44LCJRKhxApZktzVVQlVcJh+1Aftea8pcOrfTQXs= for .immoweb.be/>]>


Try to query the countries endpoint using the cookie, save the output and print it.

In [4]:
# query the /countries endpoint, assign the output to the countries variable (1 line)
countries = requests.get(countries_url, cookies=cookies).json()
# display the countries variable (1 line)
print(countries)

['us', 'be', 'fr', 'ma', 'ru']


Chances are the cookie has expired... Thanksfully, you got a nice error message. For now, simply execute the last 2 cells quickly so you get a result.

### Getting the actual data from the API

Query the `/leaders` endpoint.

In [5]:
# Set the leaders_url variable (1 line)
leaders_url = root_url + '/leaders'

# query the /leaders endpoint, assign the output to the leaders variable (1 line)
leaders = requests.get(leaders_url, cookies=cookies).json()

# display the leaders variable (1 line)
print(leaders)

{'message': 'Please specify a country'}


It looks like this endpoint requires additional information in order to return its result. Check the API [*documentation*](https://country-leaders.onrender.com/docs) in your web browser.

Change the query to accept *parameters*. You should know where to find help by now.

In [6]:
# query the /leaders endpoint using cookies and parameters (take any country in countries)
# assign the output to the leaders variable (1 line)
leaders = requests.get(leaders_url, cookies=cookies, params={'country': 'fr'}).json()

# display the leaders variable (1 line)
print(leaders)

[{'id': 'Q157', 'first_name': 'François', 'last_name': 'Hollande', 'birth_date': '1954-08-12', 'death_date': None, 'place_of_birth': 'Rouen', 'wikipedia_url': 'https://fr.wikipedia.org/wiki/Fran%C3%A7ois_Hollande', 'start_mandate': '2012-05-15', 'end_mandate': '2017-05-14'}, {'id': 'Q329', 'first_name': 'Nicolas', 'last_name': 'Sarkozy', 'birth_date': '1955-01-28', 'death_date': None, 'place_of_birth': 'Paris', 'wikipedia_url': 'https://fr.wikipedia.org/wiki/Nicolas_Sarkozy', 'start_mandate': '2007-05-16', 'end_mandate': '2012-05-15'}, {'id': 'Q2038', 'first_name': 'François', 'last_name': 'Mitterrand', 'birth_date': '1916-10-26', 'death_date': '1996-01-08', 'place_of_birth': 'Jarnac', 'wikipedia_url': 'https://fr.wikipedia.org/wiki/Fran%C3%A7ois_Mitterrand', 'start_mandate': '1981-05-21', 'end_mandate': '1995-05-17'}, {'id': 'Q2042', 'first_name': 'Charles', 'last_name': 'de Gaulle', 'birth_date': '1890-11-22', 'death_date': '1970-11-09', 'place_of_birth': 'Lille', 'wikipedia_url': 'h

### A sneak peak at the data (finally)

Look inside a few examples. Notice the dictionary keys available for each entry. You have your first example of *structured data*. This data was sanitized for your benefit, meaning it is readily exploitable without modification.

You will also notice there is a Wikipedia link for each entry. You will need to extract additional information there. This will be a case of *semi-structured* data.

The /countries endpoint returns a `list` of several country codes.

You need to loop through this list and query the /leaders endpoint for each one. Save each `json` result in a dictionary called `leaders_per_country`.

In [7]:
# 4 lines
leaders_per_country = dict()
for country in countries:
    leaders_response = requests.get(leaders_url, cookies=cookies, params={'country': country})
    leaders_per_country[country] = leaders_response.json()

In [8]:
# or 1 line
leaders_per_country = {country: requests.get(leaders_url, cookies=cookies, params={'country': country}).json() for country in countries}

In [9]:
print(leaders_per_country)

{'us': [{'id': 'Q23', 'first_name': 'George', 'last_name': 'Washington', 'birth_date': '1732-02-22', 'death_date': '1799-12-14', 'place_of_birth': 'Westmoreland County', 'wikipedia_url': 'https://en.wikipedia.org/wiki/George_Washington', 'start_mandate': '1789-04-30', 'end_mandate': '1797-03-04'}, {'id': 'Q76', 'first_name': 'Barack', 'last_name': 'Obama', 'birth_date': '1961-08-04', 'death_date': None, 'place_of_birth': 'Kapiolani Medical Center for Women and Children', 'wikipedia_url': 'https://en.wikipedia.org/wiki/Barack_Obama', 'start_mandate': '2009-01-20', 'end_mandate': '2017-01-20'}, {'id': 'Q91', 'first_name': 'Abraham', 'last_name': 'Lincoln', 'birth_date': '1809-02-12', 'death_date': '1865-04-15', 'place_of_birth': 'Sinking Spring Farm', 'wikipedia_url': 'https://en.wikipedia.org/wiki/Abraham_Lincoln', 'start_mandate': '1861-03-04', 'end_mandate': '1865-04-15'}, {'id': 'Q207', 'first_name': 'George', 'last_name': 'Bush', 'birth_date': '1946-07-06', 'death_date': None, 'plac

It is finally time to create a `get_leaders()` function for the above code. You will build on it later-on. This function takes no parameter. Inside it, you will need to:
1. define the urls
2. get the cookies
2. get the countries
3. loop over them and save their leaders in a dictionary
4. return the dictionary

In [10]:
# < 15 lines
def get_leaders():
    # 1. define the urls
    root_url = 'https://country-leaders.onrender.com'
    countries_url = root_url + '/countries'
    cookie_url = root_url + '/cookie'
    leaders_url = root_url + '/leaders'
    
    # 2. get the cookies
    cookies = requests.get(cookie_url).cookies
    
    # 3. get the countries
    countries = requests.get(countries_url, cookies=cookies).json()
    
    # 4. loop over them and save their leaders in a dictionary
    leaders_per_country = dict()
    for country in countries:
        leaders_per_country[country] = requests.get(leaders_url, cookies=cookies, params={'country': country}).json()
    
    # 5. return the dictionary
    return leaders_per_country

Test your function, save the result in the `leaders_per_country` dictionary and check its ouput.

In [11]:
# 2 lines
leaders_per_country = get_leaders()
print(leaders_per_country)

{'us': [{'id': 'Q23', 'first_name': 'George', 'last_name': 'Washington', 'birth_date': '1732-02-22', 'death_date': '1799-12-14', 'place_of_birth': 'Westmoreland County', 'wikipedia_url': 'https://en.wikipedia.org/wiki/George_Washington', 'start_mandate': '1789-04-30', 'end_mandate': '1797-03-04'}, {'id': 'Q76', 'first_name': 'Barack', 'last_name': 'Obama', 'birth_date': '1961-08-04', 'death_date': None, 'place_of_birth': 'Kapiolani Medical Center for Women and Children', 'wikipedia_url': 'https://en.wikipedia.org/wiki/Barack_Obama', 'start_mandate': '2009-01-20', 'end_mandate': '2017-01-20'}, {'id': 'Q91', 'first_name': 'Abraham', 'last_name': 'Lincoln', 'birth_date': '1809-02-12', 'death_date': '1865-04-15', 'place_of_birth': 'Sinking Spring Farm', 'wikipedia_url': 'https://en.wikipedia.org/wiki/Abraham_Lincoln', 'start_mandate': '1861-03-04', 'end_mandate': '1865-04-15'}, {'id': 'Q207', 'first_name': 'George', 'last_name': 'Bush', 'birth_date': '1946-07-06', 'death_date': None, 'plac

## Extracting data from Wikipedia

Query one of the leaders' Wikipedia urls and display its `text` (not JSON).

In [12]:
# 3 lines
wikipedia_url = leaders_per_country['fr'][0]['wikipedia_url']
wikipedia_response = requests.get(wikipedia_url)
print(wikipedia_response.text)

<!DOCTYPE html>
<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-language-alert-in-sidebar-enabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-enabled vector-feature-main-menu-pinned-disabled vector-feature-limited-width-enabled vector-feature-limited-width-content-enabled vector-feature-zebra-design-disabled" lang="fr" dir="ltr">
<head>
<meta charset="UTF-8"/>
<title>François Hollande — Wikipédia</title>
<script>document.documentElement.className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-language-alert-in-sidebar-enabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-enabled vector-feature-main-menu-pinned-disabled vector-feature-limited-width-enabled vector-feature-limited-width-content-enabled vector-f

Ouch! You get the raw HTML code of the webpage. If you try to deal with it without tools, you will be there all night. Instead, use the [beautiful soup 4](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) *external* library. You will find more info about it [here](../../2.python/2.python_advanced/05.Scraping/1.beautifulsoup_basic.ipynb) and [here](../../2.python/2.python_advanced/05.Scraping/2.beautifulsoup_advanced.ipynb)

Using the Quickstart section, start by importing the library and loading the output of your `get_text()` function.

Use the `prettify()` function and print it to take a look. You will start the actual parsing in the next step.

In [13]:
# 3 lines
from bs4 import BeautifulSoup

soup = BeautifulSoup(wikipedia_response.text, 'html.parser')

print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-language-alert-in-sidebar-enabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-enabled vector-feature-main-menu-pinned-disabled vector-feature-limited-width-enabled vector-feature-limited-width-content-enabled vector-feature-zebra-design-disabled" dir="ltr" lang="fr">
 <head>
  <meta charset="utf-8"/>
  <title>
   François Hollande — Wikipédia
  </title>
  <script>
   document.documentElement.className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-language-alert-in-sidebar-enabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-enabled vector-feature-main-menu-pinned-disabled vector-feature-limited-width-enabled vector-feature-limited-width-conten

That looks better but you need to extract the right part of the webpage: the text of the first paragraph.

It is a bit tricky because Wikipedia pages slightly differ in structure from one language to the next. We cannot simply get the text for the first HTML paragraph.

You will start by getting all the HTML paragraphs from the HTML source and saving them in the `paragraphs` variable.

Use the documentation or google the appropriate keywords.

In [14]:
# 2 lines
paragraphs = soup.find_all('p')
print(paragraphs)

[<p class="mw-empty-elt">
</p>, <p>Pour les articles homonymes, voir <a class="mw-disambig" href="/wiki/Hollande_(homonymie)" title="Hollande (homonymie)">Hollande</a>.
</p>, <p>Cet article concerne la biographie de la personne. Pour sa présidence, voir <a href="/wiki/Pr%C3%A9sidence_de_Fran%C3%A7ois_Hollande" title="Présidence de François Hollande">présidence de François Hollande</a>.
</p>, <p class="mw-empty-elt">
</p>, <p><b>François Hollande</b> <span class="API nowrap" style="font-family:'Segoe UI','DejaVu Sans','Lucida Grande','Lucida Sans Unicode','Arial Unicode MS','Hiragino Kaku Gothic Pro',sans-serif;" title="Alphabet phonétique international">[<a class="mw-redirect" href="/wiki/API_f" title="API f"><span title="[f] « f » dans « fête ».">f</span></a><a class="mw-redirect" href="/wiki/API_%CA%81" title="API ʁ"><span title="[ʁ] « r » dans « roue ».">ʁ</span></a><a class="mw-redirect" href="/wiki/API_%C9%91%CC%83" title="API ɑ̃"><span title="[ɑ̃] « an » dans « sans ».">ɑ̃</span>

If you try different urls, you might find that the paragraph you want may be at a different index each time.

That is where you need to be clever and ask yourself what would be a reliable way to identify the right index ie. which string matches only the first paragraph whatever the language...

Spend a good 30 minutes on the problem and brainstorm with your fellow learners. If you come out empty handed, ask your coach.

1. Loop over the HTML paragraphs
2. When you have identified the correct one
  * store the [text](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#output) inside the `first_paragraph` variable
  * exit the loop

In [15]:
# <10 lines
first_paragraph = ''
for paragraph in soup.find_all('p'):
    # if the paragraph contains bold text
    if paragraph.find('b'):
        # get the text
        first_paragraph = paragraph.get_text()
        # exit the loop
        break

print(first_paragraph)


François Hollande [fʁɑ̃swa ʔɔlɑ̃d][n 2] Écouter, né le 12 août 1954 à Rouen (Seine-Maritime), est un haut fonctionnaire et homme d'État français. Il est président de la République française du 15 mai 2012 au 14 mai 2017.



At this stage, you can create a function to maintain consistency in your code. We will give you its *skeleton*, you will copy the code you wrote and make it work inside a function.

Don't forget to test your function.

In [16]:
# 10 lines
# def get_first_paragraph(wikipedia_url):
#   print(wikipedia_url) # keep this for the rest of the notebook
#   [insert your code]
#   return first_paragraph

def get_first_paragraph(wikipedia_url):
    print(wikipedia_url) # keep this for the rest of the notebook
    wikipedia_response = requests.get(wikipedia_url)
    soup = BeautifulSoup(wikipedia_response.text, 'html.parser')
    first_paragraph = ''
    for paragraph in soup.find_all('p'):
        if paragraph.find('b'):
            first_paragraph = paragraph.get_text()
            break
    return first_paragraph

In [17]:
# Test: 3 lines
wikipedia_url = leaders_per_country['fr'][0]['wikipedia_url']
first_paragraph = get_first_paragraph(wikipedia_url)
print(first_paragraph)

https://fr.wikipedia.org/wiki/Fran%C3%A7ois_Hollande
François Hollande [fʁɑ̃swa ʔɔlɑ̃d][n 2] Écouter, né le 12 août 1954 à Rouen (Seine-Maritime), est un haut fonctionnaire et homme d'État français. Il est président de la République française du 15 mai 2012 au 14 mai 2017.



### Regular expressions to the rescue

Now that you have extracted the content of the first paragraph, the only thing that remains to finish your Wikipedia scraper is to sanitize the output.

Indeed some Wikipedia references, HTML code, phonetic pronunciation etc. may linger. You might find *regular expressions* handy to get rid of them and obtain pristine text. You will find some useful documentation about regular expressions [here](../../2.python/2.python_advanced/03.Regex/regex.ipynb)

Once you have one of your regex working online, try it in the cell below. 

Hints: 
* Check the `sub()` method documentation.
* Make sure to test urls in different languages. Some may look good but other do not.

In [18]:
# 3 lines
import re

first_paragraph = re.sub(r' \[.*\].*?,', ',', first_paragraph)

print(first_paragraph)


François Hollande, né le 12 août 1954 à Rouen (Seine-Maritime), est un haut fonctionnaire et homme d'État français. Il est président de la République française du 15 mai 2012 au 14 mai 2017.



Overwrite the `get_first_paragraph()` function by applying your regex to the first paragraph before returning it.

In [19]:
# 10 lines
def get_first_paragraph(wikipedia_url):
    wikipedia_response = requests.get(wikipedia_url)
    soup = BeautifulSoup(wikipedia_response.text, 'html.parser')
    first_paragraph = ''
    for paragraph in soup.find_all('p'):
        if paragraph.find('b'):
            first_paragraph = paragraph.get_text()
            break
    first_paragraph = re.sub(r' \[.*\].*?,', ',', first_paragraph)
    return first_paragraph

Come up with other regexes to capture other patterns and sanitize the outputs completely. Modify your `get_first_paragraph()` function accordingly.

In [20]:
# < 20 lines


## Putting it all together

Let's go back to your `get_leaders()` function and update it with an *inner* loop over each leader. You will query the url provided and extract the first paragraph using the `get_first_paragraph()` function you just finished. You will then update that `leader`'s dictionary and move on to the next one.

Notice, the rest of the code should not change since you modify the leader's data one by one.

In [27]:
# < 20 lines

# This simple version will crash

def get_leaders():
    # 1. get the urls
    root_url = 'https://country-leaders.onrender.com'
    status_url = root_url + '/status'
    countries_url = root_url + '/countries'
    cookie_url = root_url + '/cookie'
    leaders_url = root_url + '/leaders'

    # 2. get the cookie
    cookies = requests.get(cookie_url).cookies

    # 3. get the countries
    countries = requests.get(countries_url, cookies=cookies).json()

    # 4. get the leaders
    leaders_per_country = {}
    for country in countries:
        print(country)
        leaders_per_country[country] = requests.get(leaders_url, cookies=cookies, params={'country': country}).json()
        for leader in leaders_per_country[country]:
            wikipedia_url = leader['wikipedia_url']
            leader['first_paragraph'] = get_first_paragraph(wikipedia_url)

    # 5. return the leaders
    return leaders_per_country

In [28]:
# < 25 lines

# This more complex version will NOT CRASH

def get_leaders():
    # 1. get the urls
    root_url = 'https://country-leaders.onrender.com'
    status_url = root_url + '/status'
    countries_url = root_url + '/countries'
    cookie_url = root_url + '/cookie'
    leaders_url = root_url + '/leaders'

    # 2. get the cookie
    cookies = requests.get(cookie_url).cookies

    # 3. get the countries
    countries = requests.get(countries_url, cookies=cookies).json()

    # 4. get the leaders
    leaders_per_country = {}
    for country in countries:
        print(country)
        leaders_per_country[country] = requests.get(leaders_url, cookies=cookies, params={'country': country}).json()

    # 5. get the first paragraph for each leader
    for country, leaders in leaders_per_country.items():
        for leader in leaders:
            wikipedia_url = leader['wikipedia_url']
            leader['first_paragraph'] = get_first_paragraph(wikipedia_url)

    # 6. return the leaders
    return leaders_per_country

In [29]:
# Check the output of your function (2 lines)
leaders_per_country = get_leaders()
print(leaders_per_country)

us
be
fr
ma
ru
{'us': [{'id': 'Q23', 'first_name': 'George', 'last_name': 'Washington', 'birth_date': '1732-02-22', 'death_date': '1799-12-14', 'place_of_birth': 'Westmoreland County', 'wikipedia_url': 'https://en.wikipedia.org/wiki/George_Washington', 'start_mandate': '1789-04-30', 'end_mandate': '1797-03-04', 'first_paragraph': 'George Washington (February 22, 1732[b]\xa0– December 14, 1799) was an American military officer, statesman, and Founding Father who served as the first president of the United States from 1789 to 1797. Appointed by the Continental Congress as commander of the Continental Army, Washington led Patriot forces to victory in the American Revolutionary War and served as president of the Constitutional Convention of 1787, which created and ratified the Constitution of the United States and the American federal government. Washington has been called the "Father of his Country" for his manifold leadership in the nation\'s founding.[10]\n'}, {'id': 'Q76', 'first_name'

Does the function crash in the middle of the loop? Chances are the cookies have expired while looping over the leaders.

Modify your function with an *exception* or check if the `status_code` is a cookie error. In either case, get new cookies and query the api again.

If your code did not crash,

In [30]:
# < 25 lines

# With a condition

def get_leaders():
    # 1. get the urls
    root_url = 'https://country-leaders.onrender.com'
    status_url = root_url + '/status'
    countries_url = root_url + '/countries'
    cookie_url = root_url + '/cookie'
    leaders_url = root_url + '/leaders'

    # 2. get the cookie
    cookies = requests.get(cookie_url).cookies

    # 3. get the countries
    countries = requests.get(countries_url, cookies=cookies).json()

    # 4. get the leaders
    leaders_per_country = {}
    for country in countries:
        print(country)
        leaders_response = requests.get(leaders_url, cookies=cookies, params={'country': country})
        if leaders_response.status_code == 200:
            leaders_per_country[country] = leaders_response.json()
        else:
            # get a new cookie
            print('new cookie')
            cookies = requests.get(cookie_url).cookies
            leaders_per_country[country] = requests.get(leaders_url, cookies=cookies, params={'country': country}).json()

        for leader in leaders_per_country[country]:
            wikipedia_url = leader['wikipedia_url']
            leader['first_paragraph'] = get_first_paragraph(wikipedia_url)

    # 6. return the leaders
    return leaders_per_country


In [31]:
# < 25 lines

# Better version with an exception

def get_leaders():
    # 1. get the urls
    root_url = 'https://country-leaders.onrender.com'
    status_url = root_url + '/status'
    countries_url = root_url + '/countries'
    cookie_url = root_url + '/cookie'
    leaders_url = root_url + '/leaders'

    # 2. get the cookie
    cookies = requests.get(cookie_url).cookies

    # 3. get the countries
    countries = requests.get(countries_url, cookies=cookies).json()

    # 4. get the leaders
    leaders_per_country = {}
    for country in countries:
        print(country)
        try:
            leaders_per_country[country] = requests.get(leaders_url, cookies=cookies, params={'country': country}).json()
        except TypeError:
            # get a new cookie
            print('new cookie')
            cookies = requests.get(cookie_url).cookies
            leaders_per_country[country] = requests.get(leaders_url, cookies=cookies, params={'country': country}).json()

        for leader in leaders_per_country[country]:
            wikipedia_url = leader['wikipedia_url']
            leader['first_paragraph'] = get_first_paragraph(wikipedia_url)

    # 6. return the leaders
    return leaders_per_country

Check the output of your function again.

In [32]:
# Check the output of your function (1 line)
leaders_per_country = get_leaders()
print(leaders_per_country)

us
be
fr
ma
ru
{'us': [{'id': 'Q23', 'first_name': 'George', 'last_name': 'Washington', 'birth_date': '1732-02-22', 'death_date': '1799-12-14', 'place_of_birth': 'Westmoreland County', 'wikipedia_url': 'https://en.wikipedia.org/wiki/George_Washington', 'start_mandate': '1789-04-30', 'end_mandate': '1797-03-04', 'first_paragraph': 'George Washington (February 22, 1732[b]\xa0– December 14, 1799) was an American military officer, statesman, and Founding Father who served as the first president of the United States from 1789 to 1797. Appointed by the Continental Congress as commander of the Continental Army, Washington led Patriot forces to victory in the American Revolutionary War and served as president of the Constitutional Convention of 1787, which created and ratified the Constitution of the United States and the American federal government. Washington has been called the "Father of his Country" for his manifold leadership in the nation\'s founding.[10]\n'}, {'id': 'Q76', 'first_name'

Well done! It took a while however... Let's speed things up. The main *bottleneck* is the loop. We call on the Wikipedia website many times.

You will use the same *session* to call all the wikipedia pages. Check the *Advanced Usage* section of the Requests module's documentation.

Start by modifying the `get_first_paragraph()` function to accept a session parameter and adjust the `get()` method call.

In [33]:
# < 20 lines
def get_first_paragraph(wikipedia_url, session):
    wikipedia_response = session.get(wikipedia_url)
    soup = BeautifulSoup(wikipedia_response.text, 'html.parser')
    first_paragraph = ''
    for paragraph in soup.find_all('p'):
        if paragraph.find('b'):
            first_paragraph = paragraph.get_text()
            break
    first_paragraph = re.sub(r' \[.*\].*?,', ',', first_paragraph)
    return first_paragraph

Modify your `get_leaders()` function to make use of a single session for all the Wikipedia calls.
1. create a `Session` object outside of the loop over countries.
2. pass it to the `get_first_paragraph()` function as an argument.

In [36]:
# <25 lines
def get_leaders():
    # 1. get the urls
    root_url = 'https://country-leaders.onrender.com'
    status_url = root_url + '/status'
    countries_url = root_url + '/countries'
    cookie_url = root_url + '/cookie'
    leaders_url = root_url + '/leaders'

    # 2. get the cookie
    cookies = requests.get(cookie_url).cookies

    # 3. get the countries
    countries = requests.get(countries_url, cookies=cookies).json()

    # 4. get the leaders
    leaders_per_country = {}
    session_render = requests.Session()
    session_wikipedia = requests.Session()
    for country in countries:
        print(country)
        try:
            leaders_per_country[country] = session_render.get(leaders_url, cookies=cookies, params={'country': country}).json()
        except TypeError:
            # get a new cookie
            print('new cookie')
            cookies = requests.get(cookie_url).cookies
            leaders_per_country[country] = session_render.get(leaders_url, cookies=cookies, params={'country': country}).json()

        for leader in leaders_per_country[country]:
            wikipedia_url = leader['wikipedia_url']
            leader['first_paragraph'] = get_first_paragraph(wikipedia_url, session_wikipedia)

    # 6. return the leaders
    return leaders_per_country

Test your new functions.



In [37]:
leaders_per_country = get_leaders()
print(leaders_per_country)

us
be
fr
ma
ru
{'us': [{'id': 'Q23', 'first_name': 'George', 'last_name': 'Washington', 'birth_date': '1732-02-22', 'death_date': '1799-12-14', 'place_of_birth': 'Westmoreland County', 'wikipedia_url': 'https://en.wikipedia.org/wiki/George_Washington', 'start_mandate': '1789-04-30', 'end_mandate': '1797-03-04', 'first_paragraph': 'George Washington (February 22, 1732[b]\xa0– December 14, 1799) was an American military officer, statesman, and Founding Father who served as the first president of the United States from 1789 to 1797. Appointed by the Continental Congress as commander of the Continental Army, Washington led Patriot forces to victory in the American Revolutionary War and served as president of the Constitutional Convention of 1787, which created and ratified the Constitution of the United States and the American federal government. Washington has been called the "Father of his Country" for his manifold leadership in the nation\'s founding.[10]\n'}, {'id': 'Q76', 'first_name'

## Saving your hard work

The final step is to save the ``leaders_per_country`` dictionary in the `leaders.json` file using the [json](https://docs.python.org/3/library/json.html) module. Check out the `with` statement.

In [38]:
# 3 lines
import json

with open('leaders.json', 'w') as f:
    json.dump(leaders_per_country, f)

Make sure the file can be read back. Write the code to read the file. And check the variables are the same.

In [40]:
# 3 lines
with open('leaders.json') as f:
    leaders_per_country_copy = json.load(f)

# Check that the two dictionaries are identical.
print(leaders_per_country_copy == leaders_per_country)

True


Make a function `save(leaders_per_country)` to call this code easily.

In [41]:
# 3 lines
def save(leaders_per_country):
    with open('leaders.json', 'w') as f:
        json.dump(leaders_per_country, f)

In [42]:
# Call the function (1 line)
save(leaders_per_country)

## Tidy things up in a stand-alone python script

Congratulations! You now have a working scraper! However, your code is scattered throughout this notebook along side the tutorials. Hardly production ready...

Copy and paste what you need in a separate `leaders_scraper.py` file.
Make sure it works by calling `python3 leaders_scraper.py`

## To go further

If you want to practice scraping, you can read this section and tackle the exercise.

You have noticed the API returns very partial results for country leaders. Many are missing. Overwrite the `get_leaders()` function to get its list from Wikipedia and extract their *personal details* from the frame on the side.

Good luck!